# Importing the libraries

In [2]:
import cv2
import mediapipe as mp
import pyfirmata
import numpy as np
import time
from timer import Timer
import threading
import winsound

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

# Connecting to the Arduino Uno

In [4]:
port = "COM3"
board = pyfirmata.Arduino(port)

In [5]:
servo_pinX = board.get_pin('d:3:s') 
servo_pinY = board.get_pin('d:2:s')
servo_pinX.write(90)
servo_pinY.write(90)
laser_pin = board.digital[13]

# Helper Fucntions

In [6]:
def put_text(image, text: str, font = cv2.FONT_HERSHEY_PLAIN, color = (255, 255, 255), thickness = 2):
    cv2.putText(image, text, (50, 100), font, 2, color = color, thickness=thickness)

In [8]:
def play(file: str):
    threading.Thread(target=winsound.PlaySound, args=(file, winsound.SND_FILENAME), daemon=True).start()

play('./sounds/hostile.wav')

In [9]:
# https://stackoverflow.com/questions/37642834/opencv-how-to-calculate-the-degreesangles-of-an-object-with-its-coordinates

w = 640 # Width of the image
h = 480 # Height of the image
fov_x = 70 # Horizontal field of view of the camera
fov_y = 50 # Vertical field of view of the camera

a_x = w/2
a_y = h/2

alpha_x = fov_x/2
alpha_y = fov_y/2

Kx = np.tan(np.deg2rad(alpha_x)) / a_x
Ky = np.tan(np.deg2rad(alpha_y)) / a_y

# Testing
p = 134
b = p - h/2
beta = np.rad2deg(np.arctan(Ky * b))
round(beta + 90)

78

In [10]:
def move_servo(x: int, y: int) -> tuple:
    global servo_pinX, servo_pinY

    b_x = x - w/2
    b_y = y - h/2

    beta_x = np.rad2deg(np.arctan(Kx * b_x))
    beta_y = np.rad2deg(np.arctan(Ky * b_y))

    angle_x = round(180 - (beta_x + 90))
    angle_y = round(beta_y + 45)

    servo_pinY.write(angle_y)
    servo_pinX.write(angle_x)

    return angle_x, angle_y

# Main Function

In [13]:
cap = cv2.VideoCapture(2)

# ws, hs = 1920, 1080
# cap.set(3, ws)
# cap.set(4, hs)

servoPos = [90, 90]  # initial servo position
servo_pinX.write(90)
servo_pinY.write(70)
offsetY = 0
start_time = None

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    

    isLaserOn = True
    isCooldown = False
    isSurrendering = True
    isShooting = False

    detectedSound = False


    warn_timer = Timer(6)
    cooldown_timer = Timer(3)
    shooting_timer = Timer(3)

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Camera not found.")
            print("Please check whether the camera is connected properly.")
            break
        
        surrenderTextColor = (0, 0, 255)

        if isShooting and shooting_timer.is_time_up():
            isShooting = False
            shooting_timer.clear()
            warn_timer.clear()


        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        if results.pose_landmarks:


            # To detect surrendering pose
            left_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y
            right_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y

            right_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
            left_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y

            # Find the position to aim
            nose_y = int(
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image.shape[0] + offsetY)

            nose_x = int(
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image.shape[1])

            # nose_z = np.abs(
            #     results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].z * image.shape[0])


            if left_wrist_y < left_shoulder_y and right_wrist_y < right_shoulder_y:
                isSurrendering = True
                surrenderTextColor = (0, 255, 0)
            else:
                isSurrendering = False

            if nose_x and nose_y:

                # if not detectedSound:
                #     play('./sounds/hostile.wav')
                #     detectedSound = True


                cv2.putText(image, f'nose_x: {nose_x}', (50, 250),
                            cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)

                cv2.putText(image, f'nose_y: {nose_y}', (50, 300),
                            cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
                # cv2.putText(image, f'nose_z: {nose_z}', (50, 350),
                #             cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)


                # Translating pixel to servo angle
                trans_x, trans_y = move_servo(nose_x, nose_y)

                cv2.putText(image, f'trans_x: {trans_x}', (50, 350),
                            cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)

                cv2.putText(image, f'trans_y: {trans_y}', (50, 400),
                            cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)

                # Crosshair
                # cv2.circle(image, (nose_x, nose_y), 50, (0, 0, 255), 2)
                # cv2.circle(image, (nose_x, nose_y),
                #            15, (0, 0, 255), cv2.FILLED)
                
                if not isSurrendering and not isShooting:

                    if not warn_timer.is_timer_running():
                        warn_timer.start()
                    else:
                        if warn_timer.is_time_up():
                            isShooting = True
                            shooting_timer.start()
                        else:
                            cv2.putText(image, f'Shooting in: {round(warn_timer.get_time_left())}', (50, 100),
                                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
                elif isSurrendering:
                    isShooting = False
                    isLaserOn = True
                    detectedSound = False
                    warn_timer.clear()
                    shooting_timer.clear()


        if isShooting:
            # put_text(
            #     image,
            #     'Shooting: ' + str(round(shooting_timer.get_time_left())),
            #     color = (255, 255, 0)
            # )
            cv2.putText(image, f'Shooting: {round(shooting_timer.get_time_left())}', (50, 100),
                        cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
            # play('./sounds/gun.wav')
            isLaserOn = not isLaserOn
        else:
            isLaserOn = True
        
        laser_pin.write(isLaserOn)

        cv2.putText(image, f'Surrender: {isSurrendering}', (50, 50),
                    cv2.FONT_HERSHEY_PLAIN, 2, surrenderTextColor, 2)

        # Showing image
        # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        cv2.imshow('Camera Feed', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cv2.destroyAllWindows()
cap.release()
servo_pinX.write(90)
servo_pinY.write(130)

# Misc

In [8]:
cv2.destroyAllWindows()
cap.release()

In [ ]:
while True:
    servo_pinY.write(15)
    time.sleep(1)
    servo_pinY.write(80)
    time.sleep(1)

## Lists all the available serial ports

In [10]:
def list_ports():
    """
    Test the ports and returns a tuple with the available ports and the ones that are working.
    """
    non_working_ports = []
    dev_port = 0
    working_ports = []
    available_ports = []
    while len(non_working_ports) < 6: # if there are more than 5 non working ports stop the testing. 
        camera = cv2.VideoCapture(dev_port)
        if not camera.isOpened():
            non_working_ports.append(dev_port)
            print("Port %s is not working." %dev_port)
        else:
            is_reading, img = camera.read()
            w = camera.get(3)
            h = camera.get(4)
            if is_reading:
                print("Port %s is working and reads images (%s x %s)" %(dev_port,h,w))
                working_ports.append(dev_port)
            else:
                print("Port %s for camera ( %s x %s) is present but does not reads." %(dev_port,h,w))
                available_ports.append(dev_port)
        dev_port +=1
    return available_ports,working_ports,non_working_ports

list_ports()

Port 0 is working and reads images (480.0 x 640.0)
Port 1 is working and reads images (480.0 x 640.0)
Port 2 is working and reads images (480.0 x 640.0)
Port 3 is working and reads images (480.0 x 640.0)
Port 4 is not working.
Port 5 is not working.
Port 6 is not working.
Port 7 is not working.
Port 8 is not working.
Port 9 is not working.


([], [0, 1, 2, 3], [4, 5, 6, 7, 8, 9])